In [4]:
pip install geopy

     -------------------------------------- 119.8/119.8 kB 6.8 MB/s eta 0:00:00
     ---------------------------------------- 40.3/40.3 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
import csv
import requests
import json
from datetime import datetime, timedelta
import time

def read_csv_file(file_path):
    data = []
    with open(file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data.append(row)
    return data

def find_matching_location(data, user_location):
    for row in data:
        if row['1단계'] == user_location or row['2단계'] == user_location or row['3단계'] == user_location:
            return row
    return None

def get_grid_coordinates(matching_row):
    grid_x = matching_row['격자 X']
    grid_y = matching_row['격자 Y']
    return grid_x, grid_y

def get_grid_coordinates_from_location(file_path, user_location):
    data = read_csv_file(file_path)
    matching_row = find_matching_location(data, user_location)
    if matching_row:
        grid_x, grid_y = get_grid_coordinates(matching_row)
        return grid_x, grid_y
    else:
        return None, None

# CSV 파일 경로
file_path = './기상청위경도.csv'

# 사용자가 입력한 장소
user_location = input("장소를 입력하세요: ")

# 격자 X와 격자 Y 가져오기
grid_x, grid_y = get_grid_coordinates_from_location(file_path, user_location)

if grid_x and grid_y:
    print(f"격자 X: {grid_x}")
    print(f"격자 Y: {grid_y}")
else:
    print("일치하는 장소가 없습니다.")
    
    
def get_current_weather(grid_x, grid_y):
    api_url = "https://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtFcst"

    api_key = "jSyht54oXEynj%2B2YfWIm4rgQ%2F0BmAP9QkmXIOPwG9Oh6kXqjpMK2WwKKcZTStBj8swVyL6lEcJj555RyTRch9A%3D%3D"

    now = datetime.now()
    base_date = now.strftime("%Y%m%d")
    base_time = (now - timedelta(hours=1)).strftime("%H30")

    params = {
        "serviceKey": api_key,
        "pageNo": "1",
        "numOfRows": "100",
        "dataType": "JSON",
        "base_date": base_date,
        "base_time": base_time,
        "nx": grid_x,  
        "ny": grid_y
    }

    response = None
    data = None

    try:
        response = requests.get(api_url, params=params)
        data = json.loads(response.text)
    except:
        time.sleep(5)  # 5초 대기 후 재시도
        response = requests.get(api_url, params=params)
        data = json.loads(response.text)
    
    return data

# 현재 날씨 정보 조회
data = get_current_weather(grid_x, grid_y)

# 날씨 정보 출력
if 'response' in data and 'body' in data['response'] and 'items' in data['response']['body']:
    items = data['response']['body']['items']['item']
    for item in items:
        category = item['category']
        fcst_value = item['fcstValue']
        
        if category == 'PTY':
            pty = fcst_value
        elif category == 'SKY':
            sky = fcst_value
        elif category == 'T1H':
            t1h = fcst_value

    # '강수형태:', pty
    # '하늘상태:', sky
    # '기온:', t1h

    sky_mapping = {
        '1': '맑음',
        '3': '구름많음',
        '4': '흐림'
    }
    sky_description = sky_mapping.get(sky, '알 수 없음')

    # Map PTY code to weather description
    pty_mapping = {
        '0': '없음',
        '1': '비',
        '2': '비/눈',
        '3': '눈',
        '5': '빗방울',
        '6': '빗방울눈날림',
        '7': '눈날림',
        '4': '소나기'  # (단기) 소나기 코드 추가
    }
    pty_description = pty_mapping.get(pty, '알 수 없음')

    print('강수형태:', pty_description)
    print('하늘상태:', sky_description)
    print('기온:', t1h)
else:
    print("날씨 정보를 가져올 수 없습니다.")

장소를 입력하세요: 강남구
격자 X: 61
격자 Y: 126
강수형태: 없음
하늘상태: 구름많음
기온: 26


In [7]:
# 주류 추천 성공

import csv
import random

temperature = float(t1h)

if 6 <= temperature <= 10:
    recommendation = "소주"
elif 0 <= temperature <= 5:
    recommendation = "위스키, 꼬냑"
elif temperature >= 25:
    recommendation = "맥주"
elif 10 <= temperature <= 24:
    recommendation = "막걸리"
elif temperature < 0:
    recommendation = "와인"
else:
    recommendation = "추천 주종가 없습니다."

print("추천 주종:", recommendation)

def read_drink_data(file_path):
    drink_data = []
    with open(file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            drink_data.append(row)
    return drink_data

def recommend_drinks(drink_type, drink_data):
    recommended_drinks = []
    for drink in drink_data:
        if drink['주종'] == drink_type:
            recommended_drinks.append(drink)
    return recommended_drinks

# CSV 파일 경로
file_path = './alcohol_df.csv'

# 음료 데이터 읽어오기
drink_data = read_drink_data(file_path)

# 예시: 추천 음료가 나왔을 때 해당 주종에 해당되는 주류명 추천
if recommendation == '맥주':
    drink_type = '맥주'
elif recommendation == '와인':
    drink_type = '와인'
elif recommendation == '소주':
    drink_type = '소주'
elif recommendation == '위스키':
    drink_type = '위스키'
elif recommendation == '전통주':
    drink_type = '전통주'
else:
    print('추천 음료가 없습니다.')

if drink_type:
    recommended_drinks = recommend_drinks(drink_type, drink_data)
    if recommended_drinks:
        random.shuffle(recommended_drinks)
        recommended_drinks = recommended_drinks[:3]
        for drink in recommended_drinks:
            print('추천 주류 정보:')
            for key, value in drink.items():
                if key not in ['id', '용량'] and value:
                    print(f'{key}: {value}')
            print()
    else:
        print('해당 주종에 대한 추천 주류가 없습니다.')


추천 주종: 맥주
추천 주류 정보:
﻿id: 2038
주종: 맥주
주류명: 하이트
생산국가: 대한민국
생산자/제조사: 하이트진로
도수: 4.5
특징: 숙성에서부터 여과까지 생산 전 공정을 맥주가 얼기 직전, 영하의 온도에서 제조하여 시원하고 청량한 페일라거 맥주 본연의 맛을 극대화

추천 주류 정보:
﻿id: 2068
주종: 맥주
주류명: 클라우드
생산국가: 대한민국
생산자/제조사: 롯데칠성음료
도수: 5
특징: 100% 발효원액에 추가로 물을 타지 않은 오리지널 그래비티 공법으로 만들어 맛이 깊고 풍부한 맥주입니다. 최고 품질의 원료를 엄선해 유럽 정통 맥주의 맛을 구현했습니다.

추천 주류 정보:
﻿id: 2064
주종: 맥주
주류명: 알로하 스컬핀 IPA
생산국가: 미국
생산자/제조사: 발라스트포인트 스컬핀
도수: 7
특징: 쓴맛은 줄이고 망고, 파인애플, 구아바 등 열대과일 향을 즐길 수 있는 탁도가 높은 IPA

